# NAGuide R methods

Setup basic methods and packages used for all methods

- BiocManager could be moved to methods who are installed from BioConductor

In [1]:
packages_base_R <-
  c("BiocManager", "reshape2", "data.table", "readr", "tibble")

install_rpackage  <- function(pkg) {
  # If not installed, install the package
  if (!require(pkg, character.only = TRUE)) {
    install.packages(pkg)
    library(pkg, character.only = TRUE)
  }
  
}

# used in the large imputation function for two packages
install_bioconductor  <- function(pkg) {
  # If not installed, install the package
  if (!require(pkg, character.only = TRUE)) {
    BiocManager::install(pkg)
    library(pkg, character.only = TRUE)
  }
  
}


for (package in packages_base_R) {
  # Check if the package is already installed
  install_rpackage(pkg = package)
}


Loading required package: BiocManager



Bioconductor version '3.18' is out-of-date; the current release version '3.19'
  is available with R version '4.4'; see https://bioconductor.org/install



Loading required package: reshape2



Loading required package: data.table



Warning message:
“package ‘data.table’ was built under R version 4.3.3”



Attaching package: ‘data.table’




The following objects are masked from ‘package:reshape2’:

    dcast, melt




Loading required package: readr



Warning message:
“package ‘readr’ was built under R version 4.3.2”


Loading required package: tibble



setup can be tricky... trying to integrate as much as possible into conda environment

Copied from [NAGuideR's github](https://github.com/wangshisheng/NAguideR/blob/15ec86263d5821990ad39a8d9f378cf4d76b25fb/inst/NAguideRapp/app.R#L1705-L1849) RShiny application. Adapted to run as standalone function in context of the Snakemake workflow.

- `df` and `df1` ?
- seems quite hacky
- code is only slightly adapted from repo to run here, mainly to install packages on the fly

In [2]:
nafunctions <- function(x, method = "zero") {
  df <- df1 <- as.data.frame(x)
  method <- tolower(method)
  if (method == "zero") {
    df[is.na(df)] <- 0
  }
  else if (method == "minimum") {
    df[is.na(df)] <- min(df1, na.rm = TRUE)
  }
  else if (method == "colmedian") {
    install_rpackage('e1071')
    df <- impute(df1, what = "median")
  }
  else if (method == "rowmedian") {
    install_rpackage('e1071')
    dfx <- impute(t(df1), what = "median")
    df <- t(dfx)
  }
  else if (method == "knn_impute") {
    install_bioconductor('impute')
    data_zero1 <-
      impute.knn(as.matrix(df1),
                 k = 10,
                 rowmax = 1,
                 colmax = 1)#rowmax = 0.9, colmax = 0.9
    df <- data_zero1$data
  }
  else if (method == "seqknn") {
    if (!require(SeqKnn)) {
      install.packages("src/R_NAGuideR/SeqKnn_1.0.1.tar.gz",
                       repos = NULL,
                       type = "source")
      library(SeqKnn)
    }
    df <- SeqKNN(df1, k = 10)
  }
  else if (method == "bpca") {
    install_bioconductor('pcaMethods')
    data_zero1 <-
      pcaMethods::pca(
        as.matrix(df1),
        nPcs = ncol(df1) - 1,
        method = "bpca",
        maxSteps = 100
      )
    df <- completeObs(data_zero1)
  }
  else if (method == "svdmethod") {
    install_bioconductor('pcaMethods')
    data_zero1 <-
      pcaMethods::pca(as.matrix(df1),
                      nPcs = ncol(df1) - 1,
                      method = "svdImpute")
    df <- completeObs(data_zero1)
  }
  else if (method == "lls") {
    install_bioconductor('pcaMethods')
    data_zero1 <- llsImpute(t(df1), k = 10)
    df <- t(completeObs(data_zero1))
  }
  else if (method == "mle") {
    install_rpackage('norm')
    xxm <- as.matrix(df1)
    ss <- norm::prelim.norm(xxm)
    thx <- norm::em.norm(ss)
    norm::rngseed(123)
    df <- norm::imp.norm(ss, thx, xxm)
  }
  else if (method == "qrilc") {
    install_bioconductor("impute")
    install_bioconductor("pcaMethods")
    install_rpackage('imputeLCMD')
    xxm <- t(df1)
    data_zero1 <-
      imputeLCMD::impute.QRILC(xxm, tune.sigma = 1)[[1]]
    df <- t(data_zero1)
  }
  else if (method == "mindet") {
    install_bioconductor("impute")
    install_bioconductor("pcaMethods")
    install_rpackage('imputeLCMD')
    xxm <- as.matrix(df1)
    df <- imputeLCMD::impute.MinDet(xxm, q = 0.01)
  }
  else if (method == "minprob") {
    install_bioconductor("impute")
    install_bioconductor("pcaMethods")
    install_rpackage('imputeLCMD')
    xxm <- as.matrix(df1)
    df <-
      imputeLCMD::impute.MinProb(xxm, q = 0.01, tune.sigma = 1)
  }
  else if (method == "irm") {
    install_rpackage('VIM')
    df <- irmi(df1, trace = TRUE, imp_var = FALSE)
    rownames(df) <- rownames(df1)
  }
  else if (method == "impseq") {
    install_rpackage('rrcovNA')
    df <- impSeq(df1)
  }
  else if (method == "impseqrob") {
    install_rpackage('rrcovNA')
    data_zero1 <- impSeqRob(df1, alpha = 0.9)
    df <- data_zero1$x
  }
  else if (method == "mice-norm") {
    install_rpackage('mice')
    minum <- 5
    datareadmi <- mice(df1,
                       m = minum,
                       seed = 1234,
                       method = "norm")
    newdatareadmi <- 0
    for (i in 1:minum) {
      newdatareadmi <- complete(datareadmi, action = i) + newdatareadmi
    }
    df <- newdatareadmi / minum
    rownames(df) <- rownames(df1)
  }
  else if (method == "mice-cart") {
    install_rpackage('mice')
    minum <- 5
    datareadmi <- mice(df1,
                       m = minum,
                       seed = 1234,
                       method = "cart")
    newdatareadmi <- 0
    for (i in 1:minum) {
      newdatareadmi <- complete(datareadmi, action = i) + newdatareadmi
    }
    df <- newdatareadmi / minum
    rownames(df) <- rownames(df1)
  }
  else if (method == "trknn") {
    source('src/R_NAGuideR/Imput_funcs.r')
    # sim_trKNN_wrapper <- function(data) {
    #   result <- data %>% as.matrix %>% t %>% imputeKNN(., k=10, distance='truncation', perc=0) %>% t
    #   return(result)
    # }
    # df1x <- sim_trKNN_wrapper(t(df1))
    # df<-as.data.frame(t(df1x))
    df <-
      imputeKNN(as.matrix(df),
                k = 10,
                distance = 'truncation',
                perc = 0)
    df <- as.data.frame(df)
  }
  else if (method == "rf") {
    install_rpackage("missForest")
    data_zero1 <- missForest(
      t(df1),
      maxiter = 10,
      ntree = 20 # input$rfntrees
      ,
      mtry = floor(nrow(df1) ^ (1 / 3)),
      verbose = TRUE
    )
    df <- t(data_zero1$ximp)
  }
  else if (method == "pi") {
    width <- 0.3 # input$piwidth
    downshift <- 1.8 # input$pidownshift
    for (i in 1:ncol(df1)) {
      temp <- df1[[i]]
      if (sum(is.na(temp)) > 0) {
        temp.sd <- width * sd(temp[!is.na(temp)], na.rm = TRUE)
        temp.mean <-
          mean(temp[!is.na(temp)], na.rm = TRUE) - downshift * sd(temp[!is.na(temp)], na.rm = TRUE)
        n.missing <- sum(is.na(temp))
        temp[is.na(temp)] <-
          rnorm(n.missing, mean = temp.mean, sd = temp.sd)
        df[[i]] <- temp
      }
    }
    df
  }
  # else if(method=="grr"){
  #   library(DreamAI)
  #   df<-impute.RegImpute(data=as.matrix(df1), fillmethod = "row_mean", maxiter_RegImpute = 10,conv_nrmse = 1e-03)
  # }
  else if (method == "gms") {
    # install.packages('GMSimpute')
    if (!require(GMSimpute)) {
      install.packages(
        "src/R_NAGuideR/GMSimpute_0.0.1.1.tar.gz",
        repos = NULL,
        type = "source"
      )
      
      library(GMSimpute)
    }
    
    df <- GMS.Lasso(df1,
                    nfolds = 3,
                    log.scale = FALSE,
                    TS.Lasso = TRUE)
  }
  else if (method == "msimpute") {
    install_bioconductor("msImpute")
    df <- msImpute(as.matrix(df),
                   method = 'v2')
    df <- as.data.frame(df)
  }
  else if (method == "msimpute_mnar") {
    install_bioconductor("msImpute")
    df <-
      msImpute(as.matrix(df),
               method = 'v2-mnar',
               group = rep(1, dim(df)[2]))
    df <- as.data.frame(df)
  }
  else if (method == "gsimp") {
    options(stringsAsFactors = F)
    # dependencies parly for sourced file
    
    install_bioconductor("impute")
    install_bioconductor("pcaMethods")
    install_rpackage('imputeLCMD')
    install_rpackage("magrittr")
    install_rpackage("glmnet")
    install_rpackage("abind")
    install_rpackage("foreach")
    install_rpackage("doParallel")
    source('src/R_NAGuideR/GSimp.R')
    
    # wrapper function with data pre-processing
    pre_processing_GS_wrapper <- function(data_raw_log) {
      # samples in rows, features in columns #
      # Initialization #
      data_raw_log_qrilc <- as.data.frame(data_raw_log) %>%
        impute.QRILC() %>% extract2(1)
      # Centralization and scaling #
      data_raw_log_qrilc_sc <-
        scale_recover(data_raw_log_qrilc, method = 'scale')
      # Data after centralization and scaling #
      data_raw_log_qrilc_sc_df <- data_raw_log_qrilc_sc[[1]]
      # Parameters for centralization and scaling (for scaling recovery) #
      data_raw_log_qrilc_sc_df_param <- data_raw_log_qrilc_sc[[2]]
      # NA position #
      NA_pos <- which(is.na(data_raw_log), arr.ind = T)
      # NA introduced to log-scaled-initialized data #
      data_raw_log_sc <- data_raw_log_qrilc_sc_df
      data_raw_log_sc[NA_pos] <- NA
      # Feed initialized and missing data into GSimp imputation #
      result <-
        data_raw_log_sc %>% GS_impute(
          .,
          iters_each = 50,
          iters_all = 10,
          initial = data_raw_log_qrilc_sc_df,
          lo = -Inf,
          hi = 'min',
          n_cores = 1,
          imp_model = 'glmnet_pred'
        )
      data_imp_log_sc <- result$data_imp
      # Data recovery #
      data_imp <- data_imp_log_sc %>%
        scale_recover(., method = 'recover',
                      param_df = data_raw_log_qrilc_sc_df_param) %>%
        extract2(1)
      return(data_imp)
    }
    df <- t(df) # samples in rows, feature in columns
    df <- pre_processing_GS_wrapper(df)
    df <- t(df) # features in rows, samples in columns
    
  }
  else{
    stop(paste("Unspported methods so far: ", method))
  }
  df <- as.data.frame(df)
  df
}

## Parameters

Choose one of the available methods. 
Some methods might fail for your dataset for unknown reasons
(and the error won't always be easy to understand)
```method
method = 'ZERO'
method = 'MINIMUM'
method = 'COLMEDIAN'
method = 'ROWMEDIAN'
method = 'KNN_IMPUTE'
method = 'SEQKNN'
method = 'BPCA'
method = 'SVDMETHOD'
method = 'LLS'
method = 'MLE'
mehtod = 'LLS'
method = 'QRILC'
method = 'MINDET'
method = 'MINPROB'
method = 'IRM'
method = 'IMPSEQ'
method = 'IMPSEQROB'
method = 'MICE-NORM'
method = 'MICE-CART'
method = 'RF'
method = 'PI'
method = 'GMS'
method = 'TRKNN',
method = 'MSIMPUTE'
method = 'MSIMPUTE_MNAR'
method = 'GSIMP'
```

In [3]:
train_split = 'runs/example/data/data_wide_sample_cols.csv' # test
folder_experiment = 'runs/example/'
method = 'KNN_IMPUTE'

In [4]:
# Parameters
train_split = "runs/dev_dataset_small/proteinGroups_N50/data/data_wide_sample_cols.csv"
method = "IRM"
folder_experiment = "runs/dev_dataset_small/proteinGroups_N50"


## Dump predictions

In [5]:
df <-
  utils::read.csv(
    train_split,
    row.names = 1,
    header = TRUE,
    stringsAsFactors = FALSE
  )
df

,X2019_12_18_14_35_Q.Exactive.HF.X.Orbitrap_6070,X2019_12_19_19_48_Q.Exactive.HF.X.Orbitrap_6070,X2019_12_20_14_15_Q.Exactive.HF.X.Orbitrap_6070,X2019_12_27_12_29_Q.Exactive.HF.X.Orbitrap_6070,X2019_12_29_15_06_Q.Exactive.HF.X.Orbitrap_6070,X2019_12_29_18_18_Q.Exactive.HF.X.Orbitrap_6070,X2020_01_02_17_38_Q.Exactive.HF.X.Orbitrap_6070,X2020_01_03_11_17_Q.Exactive.HF.X.Orbitrap_6070,X2020_01_03_16_58_Q.Exactive.HF.X.Orbitrap_6070,X2020_01_03_20_10_Q.Exactive.HF.X.Orbitrap_6070,⋯,X2020_05_20_12_33_Q.Exactive.HF.X.Orbitrap_6070,X2020_05_20_15_35_Q.Exactive.HF.X.Orbitrap_6070,X2020_05_22_14_57_Q.Exactive.HF.X.Orbitrap_6070,X2020_05_22_17_43_Q.Exactive.HF.X.Orbitrap_6070,X2020_05_26_14_20_Q.Exactive.HF.X.Orbitrap_6070,X2020_05_27_13_57_Q.Exactive.HF.X.Orbitrap_6070,X2020_05_28_04_06_Q.Exactive.HF.X.Orbitrap_6070,X2020_06_01_10_22_Q.Exactive.HF.X.Orbitrap_6070,X2020_06_01_15_41_Q.Exactive.HF.X.Orbitrap_6070,X2020_06_02_09_41_Q.Exactive.HF.X.Orbitrap_6070
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
AAAS,28.34929,27.65738,28.35216,26.82554,27.40365,27.89128,25.49826,27.35187,27.61975,27.29981,⋯,27.39591,27.72123,27.80753,28.05150,27.32457,29.11944,30.08030,27.29815,27.12110,29.03787
AACS,26.13316,25.01865,23.74047,NA,26.94849,26.48102,NA,NA,25.62377,NA,⋯,25.55151,24.91598,24.71392,25.60834,26.80045,27.06147,27.37294,NA,NA,25.98913
AAMDC,NA,24.23623,NA,NA,23.86439,26.34755,NA,NA,23.52037,25.66036,⋯,NA,NA,NA,26.03032,25.51860,25.98906,26.75977,NA,NA,25.12017
AAMP,26.77693,26.27071,27.09788,NA,26.98163,27.84942,NA,25.27523,27.13555,NA,⋯,25.74060,NA,25.85700,NA,25.20408,27.99863,27.60088,28.31847,27.96802,26.94369
AAR2,27.24780,NA,27.37741,NA,26.51979,26.91696,NA,24.84589,25.97128,26.89654,⋯,25.89322,26.72637,25.69277,25.53641,27.26284,28.29443,28.02290,NA,26.70715,28.08796
AARS,32.19493,NA,32.88453,31.92642,31.84382,32.27373,30.22561,30.97931,31.41538,NA,⋯,31.67955,32.36784,32.39376,32.59986,33.17150,34.44770,34.62178,31.39810,31.95802,33.22043
AARS2,27.15258,26.53020,27.51450,26.15688,25.34211,NA,NA,NA,25.35964,25.43692,⋯,25.82221,27.01489,27.61919,27.50120,28.12450,29.08768,NA,23.91734,NA,27.45544
AASDHPPT,27.87209,28.19145,28.47564,27.63492,27.41643,27.40406,23.80134,24.89296,NA,26.81348,⋯,27.15808,27.80710,27.15258,28.50571,28.38854,30.01491,30.31603,NA,26.21932,28.75856
AATF,28.60254,27.94190,28.77087,27.85085,27.47414,28.08106,NA,25.32382,25.75001,26.20075,⋯,27.41191,27.21706,28.06625,27.84554,28.40880,29.82548,29.11389,NA,NA,28.67570


- `data.frame` does not allow abritary column names, but only valid column names...
- tibbles don't support rownames, and the imputation methods rely on normal `data.frame`s.
Save the header row for later use.

In [6]:
original_header <- colnames(readr::read_csv(
  train_split,
  n_max = 1,
  col_names = TRUE,
  skip = 0
))
feat_name <- original_header[1]
original_header[1:5]

Rows: 1 Columns: 51


── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (1): Gene Names
dbl (45): 2019_12_18_14_35_Q-Exactive-HF-X-Orbitrap_6070, 2019_12_19_19_48_Q...
lgl  (5): 2020_01_04_04_23_Q-Exactive-HF-X-Orbitrap_6070, 2020_01_04_10_03_Q...



ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


[1] "Gene Names"                                    
[2] "2019_12_18_14_35_Q-Exactive-HF-X-Orbitrap_6070"
[3] "2019_12_19_19_48_Q-Exactive-HF-X-Orbitrap_6070"
[4] "2019_12_20_14_15_Q-Exactive-HF-X-Orbitrap_6070"
[5] "2019_12_27_12_29_Q-Exactive-HF-X-Orbitrap_6070"

Uncomment to test certain methods (only for debugging, as at least one method per package is tested using Github Actions)

In [7]:
# to_test <- c(
# 'ZERO',
# 'MINIMUM',
# 'COLMEDIAN',
# 'ROWMEDIAN',
# 'KNN_IMPUTE',
# 'SEQKNN',
# 'BPCA',
# 'SVDMETHOD',
# 'LLS',
# 'MLE',
# 'LLS',
# 'QRILC',
# 'MINDET',
# 'MINPROB',
# 'IRM',
# 'IMPSEQ',
# 'IMPSEQROB',
# 'MICE-NORM',
# 'MICE-CART',
# 'RF',
# 'PI',
# 'GMS', # fails to install on Windows
# 'TRKNN',
# 'MSIMPUTE'
# 'MSIMPUTE_MNAR'
# 'GSIMP'
# )

# for (method in to_test) {
#     print(method)
#     pred <- nafunctions(df, method)
# }

Impute and save predictions with original feature and column names

In [8]:
pred <- nafunctions(df, method)
pred <- tibble::as_tibble(cbind(rownames(pred), pred))
names(pred) <- original_header
pred

Loading required package: VIM



Loading required package: colorspace



Loading required package: grid



VIM is ready to use.




Suggestions and bug-reports can be submitted at: https://github.com/statistikat/VIM/issues



...

[1] "inner loop: 1"
[1] "numeric"
[1] "numeric"
[1] "formula used: X2019_12_18_14_35_Q.Exactive.HF.X.Orbitrap_6070 ~ X2019_12_19_19_48_Q.Exactive.HF.X.Orbitrap_6070+X2019_12_20_14_15_Q.Exactive.HF.X.Orbitrap_6070+X2019_12_27_12_29_Q.Exactive.HF.X.Orbitrap_6070+X2019_12_29_15_06_Q.Exactive.HF.X.Orbitrap_6070+X2019_12_29_18_18_Q.Exactive.HF.X.Orbitrap_6070+X2020_01_02_17_38_Q.Exactive.HF.X.Orbitrap_6070+X2020_01_03_11_17_Q.Exactive.HF.X.Orbitrap_6070+X2020_01_03_16_58_Q.Exactive.HF.X.Orbitrap_6070+X2020_01_03_20_10_Q.Exactive.HF.X.Orbitrap_6070+X2020_01_04_04_23_Q.Exactive.HF.X.Orbitrap_6070+X2020_01_04_10_03_Q.Exactive.HF.X.Orbitrap_6070+X2020_01_04_14_59_Q.Exactive.HF.X.Orbitrap_6070+X2020_01_06_20_17_Q.Exactive.HF.X.Orbitrap_6070+X2020_01_08_16_43_Q.Exactive.HF.X.Orbitrap_6070+X2020_01_09_11_07_Q.Exactive.HF.X.Orbitrap_6070+X2020_01_15_13_56_Q.Exactive.HF.X.Orbitrap_6070+X2020_01_20_15_10_Q.Exactive.HF.X.Orbitrap_6070+X2020_02_05_20_55_Q.Exactive.HF.X.Orbitrap_6070+X2020_02_10_15_41_Q

[1] "it = 24 ,  Wert = 4.84495391125159"
[1] "eps 5"
[1] "test: FALSE"
[1] "4.84495391125159 < 5 = eps"
[1] "      --> finished after 24 iterations"


Imputation performed on the following data set:




                                                type      #missing
X2019_12_18_14_35_Q.Exactive.HF.X.Orbitrap_6070 "numeric" "969"   
X2019_12_19_19_48_Q.Exactive.HF.X.Orbitrap_6070 "numeric" "1120"  
X2019_12_20_14_15_Q.Exactive.HF.X.Orbitrap_6070 "numeric" "872"   
X2019_12_27_12_29_Q.Exactive.HF.X.Orbitrap_6070 "numeric" "1113"  
X2019_12_29_15_06_Q.Exactive.HF.X.Orbitrap_6070 "numeric" "856"   
X2019_12_29_18_18_Q.Exactive.HF.X.Orbitrap_6070 "numeric" "768"   
X2020_01_02_17_38_Q.Exactive.HF.X.Orbitrap_6070 "numeric" "1872"  
X2020_01_03_11_17_Q.Exactive.HF.X.Orbitrap_6070 "numeric" "1794"  
X2020_01_03_16_58_Q.Exactive.HF.X.Orbitrap_6070 "numeric" "1103"  
X2020_01_03_20_10_Q.Exactive.HF.X.Orbitrap_6070 "numeric" "1097"  
X2020_01_04_04_23_Q.Exactive.HF.X.Orbitrap_6070 "numeric" "911"   
X2020_01_04_10_03_Q.Exactive.HF.X.Orbitrap_6070 "numeric" "2252"  
X2020_01_04_14_59_Q.Exactive.HF.X.Orbitrap_6070 "numeric" "1902"  
X2020_01_06_20_17_Q.Exactive.HF.X.Orbitrap_6070 "numeric" "876

Gene Names,2019_12_18_14_35_Q-Exactive-HF-X-Orbitrap_6070,2019_12_19_19_48_Q-Exactive-HF-X-Orbitrap_6070,2019_12_20_14_15_Q-Exactive-HF-X-Orbitrap_6070,2019_12_27_12_29_Q-Exactive-HF-X-Orbitrap_6070,2019_12_29_15_06_Q-Exactive-HF-X-Orbitrap_6070,2019_12_29_18_18_Q-Exactive-HF-X-Orbitrap_6070,2020_01_02_17_38_Q-Exactive-HF-X-Orbitrap_6070,2020_01_03_11_17_Q-Exactive-HF-X-Orbitrap_6070,2020_01_03_16_58_Q-Exactive-HF-X-Orbitrap_6070,⋯,2020_05_20_12_33_Q-Exactive-HF-X-Orbitrap_6070,2020_05_20_15_35_Q-Exactive-HF-X-Orbitrap_6070,2020_05_22_14_57_Q-Exactive-HF-X-Orbitrap_6070,2020_05_22_17_43_Q-Exactive-HF-X-Orbitrap_6070,2020_05_26_14_20_Q-Exactive-HF-X-Orbitrap_6070,2020_05_27_13_57_Q-Exactive-HF-X-Orbitrap_6070,2020_05_28_04_06_Q-Exactive-HF-X-Orbitrap_6070,2020_06_01_10_22_Q-Exactive-HF-X-Orbitrap_6070,2020_06_01_15_41_Q-Exactive-HF-X-Orbitrap_6070,2020_06_02_09_41_Q-Exactive-HF-X-Orbitrap_6070
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
AAAS,28.34929,27.65738,28.35216,26.82554,27.40365,27.89128,25.49826,27.35187,27.61975,⋯,27.39591,27.72123,27.80753,28.05150,27.32457,29.11944,30.08030,27.29815,27.12110,29.03787
AACS,26.13316,25.01865,23.74047,23.08471,26.94849,26.48102,24.66107,24.59417,25.62377,⋯,25.55151,24.91598,24.71392,25.60834,26.80045,27.06147,27.37294,26.09570,26.76520,25.98913
AAMDC,24.72050,24.23623,24.83296,23.75698,23.86439,26.34755,24.03260,23.51993,23.52037,⋯,24.18769,23.99458,24.16063,26.03032,25.51860,25.98906,26.75977,24.90002,25.91867,25.12017
AAMP,26.77693,26.27071,27.09788,26.08710,26.98163,27.84942,25.42675,25.27523,27.13555,⋯,25.74060,27.10810,25.85700,25.10049,25.20408,27.99863,27.60088,28.31847,27.96802,26.94369
AAR2,27.24780,26.05693,27.37741,26.23949,26.51979,26.91696,24.93173,24.84589,25.97128,⋯,25.89322,26.72637,25.69277,25.53641,27.26284,28.29443,28.02290,25.77508,26.70715,28.08796
AARS,32.19493,32.14166,32.88453,31.92642,31.84382,32.27373,30.22561,30.97931,31.41538,⋯,31.67955,32.36784,32.39376,32.59986,33.17150,34.44770,34.62178,31.39810,31.95802,33.22043
AARS2,27.15258,26.53020,27.51450,26.15688,25.34211,25.78359,22.46734,22.93201,25.35964,⋯,25.82221,27.01489,27.61919,27.50120,28.12450,29.08768,30.20580,23.91734,24.26821,27.45544
AASDHPPT,27.87209,28.19145,28.47564,27.63492,27.41643,27.40406,23.80134,24.89296,28.36660,⋯,27.15808,27.80710,27.15258,28.50571,28.38854,30.01491,30.31603,26.93907,26.21932,28.75856
AATF,28.60254,27.94190,28.77087,27.85085,27.47414,28.08106,24.40470,25.32382,25.75001,⋯,27.41191,27.21706,28.06625,27.84554,28.40880,29.82548,29.11389,26.94907,26.62394,28.67570


In [9]:
pred <- reshape2::melt(pred, id.vars = feat_name)
names(pred) <- c(feat_name, 'Sample ID', method)
pred <- pred[reshape2::melt(is.na(df))['value'] == TRUE, ]
pred

,Gene Names,Sample ID,IRM
,<chr>,<fct>,<dbl>
3,AAMDC,2019_12_18_14_35_Q-Exactive-HF-X-Orbitrap_6070,24.72050
12,ABCB7,2019_12_18_14_35_Q-Exactive-HF-X-Orbitrap_6070,25.10741
14,ABCC4,2019_12_18_14_35_Q-Exactive-HF-X-Orbitrap_6070,23.26840
15,ABCD1,2019_12_18_14_35_Q-Exactive-HF-X-Orbitrap_6070,26.75319
17,ABCE1,2019_12_18_14_35_Q-Exactive-HF-X-Orbitrap_6070,29.75096
26,ABI2,2019_12_18_14_35_Q-Exactive-HF-X-Orbitrap_6070,23.15326
28,ABRACL,2019_12_18_14_35_Q-Exactive-HF-X-Orbitrap_6070,26.10146
39,ACIN1,2019_12_18_14_35_Q-Exactive-HF-X-Orbitrap_6070,30.24177
41,ACO1,2019_12_18_14_35_Q-Exactive-HF-X-Orbitrap_6070,27.56911


In [10]:
dim(pred)

[1] 52352     3

In [11]:
fname = file.path(folder_experiment,
                  'preds',
                  paste0('pred_all_', toupper(method), '.csv'))
fname

[1] "runs/dev_dataset_small/proteinGroups_N50/preds/pred_all_IRM.csv"

In [12]:
write_csv(pred, path = fname)

Warning message:
“The `path` argument of `write_csv()` is deprecated as of readr 1.4.0.
ℹ Please use the `file` argument instead.”
